# Amazon SageMaker Model Governance - Model Cards


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker_model_governance|model_card.ipynb)

---

This notebook walks you through the features of Amazon SageMaker Model Cards. For more information, see [Model Cards](https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards.html) in the _Amazon SageMaker Developer Guide_.

Amazon SageMaker Model Cards give you the ability to create a centralized, customizable fact-sheet to document critical details about your machine learning (ML) models. Use model cards to keep a record of model information, such as intended uses, risk ratings, training details, evaluation metrics, and more for streamlined governance and reporting. 

In this example, you create a binary classification model along with a model card to document model details along the way. Learn how to create, read, update, delete, and export model cards using the Amazon SageMaker Python SDK.

---
## Contents

1. [Setup](#Setup)
1. [Prepare a Binary Classification Model](#Model)
1. [Create Model Card](#ModelCard)
1. [Update Model Card](#Update)
1. [Load Model Card](#Load)
1. [List Model Card History](#ListHistory)
1. [Export Model Card](#Export)
1. [Cleanup](#Cleanup)

---
## Setup
To begin, you must specify the following information:
- The IAM role ARN used to give SageMaker training and hosting access to your data. The following example uses the SageMaker execution role.
- The SageMaker session used to manage interactions with Amazon SageMaker Model Card API methods.
- The S3 URI (`bucket` and `prefix`) where you want to store training artifacts, models, and any exported model card PDFs. This S3 bucket should be in the same Region as your Notebook Instance, training, and hosting configurations. The following example uses the default SageMaker S3 bucket and creates a default SageMaker S3 bucket if one does not already exist.
- The S3 session used to manage interactions with Amazon S3 storage.

In [1]:
! pip install --upgrade sagemaker

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info/METADATA'



In [2]:
import boto3
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker_session = Session()

bucket = sagemaker_session.default_bucket()
prefix = "from-idea-to-prod-model-card"

region = sagemaker_session.boto_region_name
s3 = boto3.client("s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Next, import the necessary Python libraries.

In [3]:
import io
import os
import numpy as np
from six.moves.urllib.parse import urlparse
from pprint import pprint
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
import sagemaker.amazon.common as smac
from sagemaker.model_card import (
    ModelCard,
    ModelOverview,
    ObjectiveFunction,
    Function,
    TrainingDetails,
    IntendedUses,
    BusinessDetails,
    EvaluationJob,
    AdditionalInformation,
    Metric,
    MetricGroup,
    ModelCardStatusEnum,
    ObjectiveFunctionEnum,
    FacetEnum,
    RiskRatingEnum,
    MetricTypeEnum,
    EvaluationMetricTypeEnum,
)

### 1. Create Estimator for training job
Model already trained in SageMaker 

In [6]:
#setup training estimator
training_job_name = '<<enter your training job name>>'
estimator = sagemaker.estimator.Estimator.attach(training_job_name)
s3uri_model = estimator.model_data
print("\ns3uri_model =",s3uri_model)



2024-04-09 17:41:33 Starting - Preparing the instances for training
2024-04-09 17:41:33 Downloading - Downloading the training image
2024-04-09 17:41:33 Training - Training image download completed. Training in progress.
2024-04-09 17:41:33 Uploading - Uploading generated training model
2024-04-09 17:41:33 Completed - Training job completed

s3uri_model = s3://sagemaker-us-east-1-624178040188/from-idea-to-prod/xgboost/output/from-idea-to-prod-train-g7gwxxamjlaz-AHi4F06BJy/output/model.tar.gz


### 2. Create a model

In [7]:
model_name = "from-idea-to-prod-model-<<enter version i.e. V1>>"
model = estimator.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)
print(f"Model name: {model_name}")

INFO:sagemaker:Creating model with name: from-idea-to-prod-model


Model name: from-idea-to-prod-model


---
## Create Model Card<a name="ModelCard"></a>
Document your binary classification model details in an Amazon SageMaker Model Card using the SageMaker Python SDK.

### 1. Auto-collect model details
Automatically collect basic model information like model ID, training environment, and the model output S3 URI. Add additional model information such as a description, problem type, algorithm type, model creator, and model owner.

In [8]:
model_overview = ModelOverview.from_model_name(
    model_name=model_name,
    sagemaker_session=sagemaker_session,
    model_description="This is an example binary classification model used for a Python SDK demo of Amazon SageMaker Model Cards.",
    problem_type="Binary Classification",
    algorithm_type="Logistic Regression",
    model_creator="DEMO-ModelCard",
    model_owner="DEMO-ModelCard",
)
print(f"Model id: {model_overview.model_id}")
print(f"Model training images: {model_overview.inference_environment.container_image}")
print(f"Model: {model_overview.model_artifact}")

Model id: arn:aws:sagemaker:us-east-1:624178040188:model/from-idea-to-prod-model
Model training images: ['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1']
Model: ['s3://sagemaker-us-east-1-624178040188/from-idea-to-prod/xgboost/output/from-idea-to-prod-train-g7gwxxamjlaz-AHi4F06BJy/output/model.tar.gz']


### 2. Auto-collect training details
Automatically collect basic training information like training ID, training environment, and training metrics. Add additional training information such as objective function details and training observations.

In [9]:
objective_function = ObjectiveFunction(
    function=Function(
        function=ObjectiveFunctionEnum.MINIMIZE,
        facet=FacetEnum.LOSS,
    ),
    notes="This is an example objective function.",
)
training_details = TrainingDetails.from_model_overview(
    model_overview=model_overview,
    sagemaker_session=sagemaker_session,
    objective_function=objective_function,
    training_observations="Add model training observations here.",
)
print(f"Training job id: {training_details.training_job_details.training_arn}")
print(
    f"Training image: {training_details.training_job_details.training_environment.container_image}"
)
print("Training Metrics: ")
pprint(
    [
        {"name": i.name, "value": i.value}
        for i in training_details.training_job_details.training_metrics
    ]
)

Training job id: arn:aws:sagemaker:us-east-1:624178040188:training-job/from-idea-to-prod-train-g7gwxxamjlaz-AHi4F06BJy
Training image: ['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1']
Training Metrics: 
[{'name': 'validation:auc', 'value': 0.771399974822998},
 {'name': 'train:auc', 'value': 0.7876999974250793}]


### 3. Collect evaluation details
Add evaluation observations, datasets, and metrics.

In [10]:
manual_metric_group = MetricGroup(
    name="binary classification metrics",
    metric_data=[Metric(name="accuracy", type=MetricTypeEnum.NUMBER, value=0.5)],
)
example_evaluation_job = EvaluationJob(
    name="Example evaluation job",
    evaluation_observation="Evaluation observations.",
    datasets=["s3://sagemaker-us-east-1-624178040188/from-idea-to-prod/xgboost/evaluation/evaluation.json"],
    metric_groups=[manual_metric_group],
)
evaluation_details = [example_evaluation_job]

#### (Optional) 3.1 Parse metrics from existing evaluation report
If you have existing evaluation reports generated by [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-run.html) or [SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html), upload them to S3 and provide an S3 URI to automatically parse evaluation metrics. To add your own generic model card evaluation report, provide a report in the [evaluation results JSON format](https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards-json-schema.html). See the example JSON files in the `./example_metrics` folder for reference.
##### Collect metrics from a JSON format evaluation report

In [11]:
report_type = "clarify_bias.json"
example_evaluation_job.add_metric_group_from_json(
    f"example_metrics/{report_type}", EvaluationMetricTypeEnum.CLARIFY_BIAS
)

##### Collect metrics from S3

In [12]:
# upload metric file to s3
with open(f"example_metrics/{report_type}", "rb") as metrics:
    s3.upload_fileobj(
        metrics,
        Bucket=bucket,
        Key=f"{prefix}/{report_type}",
        ExtraArgs={"ContentType": "application/json"},
    )

metric_s3_url = f"s3://{bucket}/{prefix}/{report_type}"
example_evaluation_job.add_metric_group_from_s3(
    session=sagemaker_session.boto_session,
    s3_url=metric_s3_url,
    metric_type=EvaluationMetricTypeEnum.CLARIFY_BIAS,
)

### 4. Collect additional details
Add the intended uses of your model and business details and any additional information that you want to include in your model card. For more information on intended uses and business details, see [Model Cards](https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards.html) in the `Amazon SageMaker Developer Guide`.

In [13]:
intended_uses = IntendedUses(
    purpose_of_model="Test model card.",
    intended_uses="Not used except this test.",
    factors_affecting_model_efficiency="No.",
    risk_rating=RiskRatingEnum.LOW,
    explanations_for_risk_rating="Just an example.",
)
business_details = BusinessDetails(
    business_problem="The business problem that your model is used to solve.",
    business_stakeholders="The stakeholders who have the interest in the business that your model is used for.",
    line_of_business="Services that the business is offering.",
)
additional_information = AdditionalInformation(
    ethical_considerations="Your model ethical consideration.",
    caveats_and_recommendations="Your model's caveats and recommendations.",
    custom_details={"custom details1": "details value"},
)

### 5. Initialize a model card
Initialize a model card with the information collected in the previous steps.

In [14]:
model_card_name = "from-idea-to-prod-model-card"
my_card = ModelCard(
    name=model_card_name,
    status=ModelCardStatusEnum.DRAFT,
    model_overview=model_overview,
    training_details=training_details,
    intended_uses=intended_uses,
    business_details=business_details,
    evaluation_details=evaluation_details,
    additional_information=additional_information,
    sagemaker_session=sagemaker_session,
)
my_card.create()
print(f"Model card {my_card.name} is successfully created with id {my_card.arn}")

INFO:sagemaker.model_card.model_card:Creating model card with name: from-idea-to-prod-model-card


Model card from-idea-to-prod-model-card is successfully created with id arn:aws:sagemaker:us-east-1:624178040188:model-card/from-idea-to-prod-model-card


---
## Update Model Card<a name="Update"></a>
After creating a model card, you can update the model card information. Updating a model card creates a new model card version.

In [15]:
my_card.model_overview.model_description = "the model is updated."
my_card.update()

INFO:sagemaker.model_card.model_card:Updating model card: from-idea-to-prod-model-card
INFO:sagemaker.model_card.model_card:Update model card content


{'content': {'ModelCardArn': 'arn:aws:sagemaker:us-east-1:624178040188:model-card/from-idea-to-prod-model-card',
  'ResponseMetadata': {'RequestId': '013b5db2-59b3-49e3-af8e-bc44292a09ca',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '013b5db2-59b3-49e3-af8e-bc44292a09ca',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '99',
    'date': 'Tue, 09 Apr 2024 18:18:48 GMT'},
   'RetryAttempts': 0}}}

---
## Load Model Card<a name="Load"></a>
Load an existing model card with the model card name.

In [16]:
my_card2 = ModelCard.load(
    name=model_card_name,
    sagemaker_session=sagemaker_session,
)
print(f"Model id: {my_card2.arn}")
print(f"Model description: {my_card.model_overview.model_description}")

INFO:sagemaker.model_card.model_card:Load model card: from-idea-to-prod-model-card


Model id: arn:aws:sagemaker:us-east-1:624178040188:model-card/from-idea-to-prod-model-card
Model description: the model is updated.


---
## List Model Card History<a name="ListHistory"></a>
Track the model card history by listing historical versions.

In [17]:
history = my_card.get_version_history()
assert len(history) == 2  # one for creation and one for update

---
## Export Model Card<a name="Export"></a>
Share the model card by exporting it to a PDF file.

### 1. Create an export job

In [18]:
s3_output_path = f"s3://{bucket}/{prefix}/export"
pdf_s3_url = my_card.export_pdf(s3_output_path=s3_output_path)

INFO:sagemaker.model_card.model_card:Model card from-idea-to-prod-model-card is successfully exported to s3://sagemaker-us-east-1-624178040188/from-idea-to-prod-model-card/export/from-idea-to-prod-model-card/from-idea-to-prod-model-card-1712686741-041d.pdf.


### (optional) List export jobs
Check all the export jobs for this model card.

In [19]:
my_card.list_export_jobs()

{'ModelCardExportJobSummaries': [{'ModelCardExportJobName': 'from-idea-to-prod-model-card-1712686741-041d',
   'ModelCardExportJobArn': 'arn:aws:sagemaker:us-east-1:624178040188:model-card/from-idea-to-prod-model-card/export-job/from-idea-to-prod-model-card-1712686741-041d',
   'Status': 'Completed',
   'ModelCardName': 'from-idea-to-prod-model-card',
   'ModelCardVersion': 0,
   'CreatedAt': datetime.datetime(2024, 4, 9, 18, 19, 1, 766000, tzinfo=tzlocal()),
   'LastModifiedAt': datetime.datetime(2024, 4, 9, 18, 19, 2, 997000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '3c132b69-bdb2-44bb-ba87-608d5415760b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3c132b69-bdb2-44bb-ba87-608d5415760b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '422',
   'date': 'Tue, 09 Apr 2024 18:19:13 GMT'},
  'RetryAttempts': 0}}

### 2. Download the exported model card PDF
The downloaded PDF is stored in the same directory as this notebook by default.

#### Parse the bucket and key of the exported PDF

In [20]:
parsed_url = urlparse(pdf_s3_url)
pdf_bucket = parsed_url.netloc
pdf_key = parsed_url.path.lstrip("/")

#### Download

In [21]:
file_name = parsed_url.path.split("/")[-1]
s3.download_file(Filename=file_name, Bucket=pdf_bucket, Key=pdf_key)
print(f"{file_name} is downloaded to \n{os.path.join(os.getcwd(), file_name)}")

from-idea-to-prod-model-card-1712686741-041d.pdf is downloaded to 
/home/sagemaker-user/amazon-sagemaker-examples/sagemaker_model_governance/from-idea-to-prod-model-card-1712686741-041d.pdf


---
## Cleanup<a name="Cleanup"></a>
Delete the following resources:
1. The model card
2. The exported model card PDF
3. The example binary classification model

In [ ]:
my_card.delete()

s3.delete_object(Bucket=pdf_bucket, Key=pdf_key)

sagemaker_session.delete_model(model_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker_model_governance|model_card.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker_model_governance|model_card.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker_model_governance|model_card.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker_model_governance|model_card.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker_model_governance|model_card.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker_model_governance|model_card.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker_model_governance|model_card.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker_model_governance|model_card.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker_model_governance|model_card.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker_model_governance|model_card.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker_model_governance|model_card.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker_model_governance|model_card.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker_model_governance|model_card.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker_model_governance|model_card.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker_model_governance|model_card.ipynb)
